<a href="https://colab.research.google.com/github/sinajahangir/Hyperparameter-LSTM-TF/blob/main/Tensorflow_ED_Wavelet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This Colab notebook showcases the training/deplyment of encoder-decoder (ED) model for multi-step ahead hydrological forecasting.

The paper explaining the methodology can be accessed via:

https://www.sciencedirect.com/science/article/abs/pii/S0022169423002111

Bayesian optimization is used for hyperparamter optimization of the ED model

The models are trained using pinball loss for probabilistic inference

CAMELS data can be obtained at: https://gdex.ucar.edu/dataset/camels.html

Reference:
A. J. Newman, M. P. Clark, K. Sampson, A. Wood, L. E. Hay, A. Bock, R. J. Viger, D. Blodgett, L. Brekke, J. R. Arnold, T. Hopson, and Q. Duan: Development of a large-sample watershed-scale hydrometeorological dataset for the contiguous USA: dataset characteristics and assessment of regional variability in hydrologic model performance. Hydrol. Earth Syst. Sci., 19, 209-223, doi:10.5194/hess-19-209-2015, 2015.

# Import libraries

In [1]:
!pip install scikit-optimize #This is used for Bayesian optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.4
    Uninstalling typeguard-4.4.4:
      Successfully uninstalled typeguard-4.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [2]:
"""
Loading necessary libraries
"""
#importing necessary libs
import os
import datetime
import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import files
from os import chdir

In [3]:
"""
Loading necessary libraries
"""
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args
from tensorflow import keras

In [16]:
"""
Loading necessary libraries
"""
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils import plot_model

# Error metrics

In [18]:
def RMSE(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    rmse=(np.nanmean(((Pr-Y)**2)))**0.5
    return rmse

def MAE(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    error=Y-Pr
    return np.nanmean(abs(error))

def CC(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    return pearsonr(Pr.flatten(),Y.flatten())[0]

def BIAS(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    error=Y-Pr
    return np.nanmean(error)

def PBIAS(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    error=Y-Pr
    return (np.nansum(error)/np.nansum(Y))*100

def NSE(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    error=Y-Pr
    nse=1-(np.nansum((error)**2))/np.nansum((Y-np.nanmean(Y))**2)
    return nse
def nRMSE(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    rmse=(np.nanmean(((Pr-Y)**2)))**0.5
    nrmse=rmse/np.nanmean(Y)
    return nrmse
def MAPE(Pr,Y):
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    error=(Y-Pr)/(Y)
    error[Y==0]=1
    mape=np.nansum(np.abs(error))/len(error)
    return mape

# Load data

You have to mount Google Drive

Assuming data is located in ED_Wavelet (change this)

In [7]:
chdir(r'/content/drive/MyDrive/ED_Wavelet') #change this
import CamelsFunctions as cf #for pre-processing
from sklearn.preprocessing import StandardScaler #normalization/scaling the input/output
scaler_x = StandardScaler()
scaler_x_dec=StandardScaler() #decoder
scaler_y=StandardScaler()

In [8]:
#iid=14154500;09081600;03488000 cathcments of interest
iid='14154500'
n_steps_in=180 #lookback period
n_steps_out=7 # lead time
df=pd.read_csv('best_%s_lead%d_data.csv'%(iid,n_steps_out),index_col=0) #read csv file

col_names=df.columns #obtain all column names
q_dec=[]
for ii in col_names:
    if ii[0]=='Q':
        q_dec.append(ii) #all q related columns

df_input=df.drop(labels='Lagged.Q.m3.s', axis=1) #This should be removed
df_Q=df_input[['Q.m3.s']]
df_dec=df_input[q_dec]
df_enc=df_input.iloc[:,:5] #encoder input

arr_q=np.asarray(df_Q) #observation
data_cat=np.asarray(df_enc)
data_cat_dec=np.asarray(df_dec)

In [9]:
#Training data
data_X=data_cat[:int(0.7*len(arr_q))]
data_Y=arr_q[:int(0.7*len(arr_q))]
data_X_dec=data_cat_dec[:int(0.7*len(arr_q))]
  #Transform
data_X_tr=scaler_x.fit_transform(data_X)
data_X_tr_dec=scaler_x_dec.fit_transform(data_X_dec)
data_Y_tr=scaler_y.fit_transform(data_Y)

#Validation data
data_Xval=data_cat[int(0.7*len(arr_q)):int(0.85*len(arr_q))]
data_Xval_dec=data_cat_dec[int(0.7*len(arr_q)):int(0.85*len(arr_q))]
data_Yval=arr_q[int(0.7*len(arr_q)):int(0.85*len(arr_q))]
  #Transform
data_Xval_tr=scaler_x.transform(data_Xval)
data_Xval_tr_dec=scaler_x_dec.transform(data_Xval_dec)
data_Yval_tr=scaler_y.transform(data_Yval)

#Test data
data_Xtest=data_cat[int(0.85*len(arr_q)):]
data_Xtest_dec=data_cat_dec[int(0.85*len(arr_q)):]
data_Ytest=arr_q[int(0.85*len(arr_q)):]

  #Transform
data_Xtest_tr=scaler_x.transform(data_Xtest)
data_Xtest_tr_dec=scaler_x_dec.transform(data_Xtest_dec)
data_Ytest_tr=scaler_y.transform(data_Ytest)

In [10]:
xtrain,ytrain=cf.split_sequence_multi_train(data_X_tr,data_Y_tr,n_steps_in,n_steps_out,mode='seq')
xval,yval=cf.split_sequence_multi_train(data_Xval_tr,data_Yval_tr,n_steps_in,n_steps_out,mode='seq')
xtest,ytest=cf.split_sequence_forecast(data_Xtest_tr,data_Ytest_tr,n_steps_in,n_steps_out,mode='seq')

xtrain_dec,ytrain_dec=cf.split_sequence_multi_train(data_X_tr_dec,data_Y_tr,n_steps_in,n_steps_out,mode='seq')
xval_dec,yval_dec=cf.split_sequence_multi_train(data_Xval_tr_dec,data_Yval_tr,n_steps_in,n_steps_out,mode='seq')
xtest_dec,ytest_dec=cf.split_sequence_forecast(data_Xtest_tr_dec,data_Ytest_tr,n_steps_in,n_steps_out,mode='seq')

# Model development

In [11]:
def pinball_loss(y_true, y_pred, tau):
    error = y_true - y_pred
    loss = tf.where(error >= 0, tau * error, (tau - 1) * error)
    return tf.reduce_mean(loss)

In [27]:
def model_builder_ED(lstm_out,dense_out1=32,dropout1=0.1,activation_dense_1='relu',\
                     nout=n_steps_out,quantile=0.5,n_features=np.shape(xtrain)[-1],n_features_dec=np.shape(xtrain_dec)[-1]):
  n_total_features=n_features
  n_deterministic_features=n_features_dec
  window_len=n_steps_in
  latent_dim_lstm=int(lstm_out) # Explicitly cast to integer
  latent_dim_dense=int(latent_dim_lstm/2)
  forecast_len=n_steps_in
# First branch of the net is an lstm which embeds the past
  past_inputs = tf.keras.Input(
    shape=(window_len, n_total_features), name='past_inputs')
# Encoding the past
  encoder = tf.keras.layers.LSTM(latent_dim_lstm, return_state=True)
  encoder_outputs, state_h, state_c = encoder(past_inputs)

  future_inputs = tf.keras.Input(
    shape=(forecast_len, n_deterministic_features), name='future_inputs')
# Combining future inputs with recurrent branch output
  decoder_lstm = tf.keras.layers.LSTM(latent_dim_lstm,recurrent_regularizer=tf.keras.regularizers.l2(0.001),return_sequences=False)
  x = decoder_lstm(future_inputs,initial_state=[state_h, state_c])
  x = tf.keras.layers.Dense(latent_dim_dense,activation=activation_dense_1)(x)
  x = tf.keras.layers.Dropout(dropout1)(x)
  output = tf.keras.layers.Dense(n_steps_out, activation='linear')(x)
  ED_model = tf.keras.models.Model(inputs=[past_inputs, future_inputs], outputs=output)
  ED_model.compile(optimizer=Adam(learning_rate=1e-4),
                loss= lambda y_true, y_pred: pinball_loss(y_true, y_pred, tau=quantile),
                metrics=[tf.losses.MeanAbsoluteError()])
  return ED_model

In [ ]:
# example for model structure
model_temp=model_builder_ED(lstm_out=128,dense_out1=64)
plot_model(model_temp,show_shapes=True,)

# Bayesian optimization

In [29]:
#search space
dim_lstm_out = Integer(low=32, high=256, name='lstm_out')
dim_dense_out1= Integer(low=16, high=128, name='dense_out1')
dim_dropout1= Real(low=0, high=0.3, prior='uniform' ,name='dropout1')
dim_dense_active1=Categorical(['relu','tanh','sigmoid'],name='activation_dense_1')

In [30]:
dimensions = [dim_lstm_out,dim_dense_out1,dim_dropout1,dim_dense_active1]

In [33]:
@use_named_args(dimensions=dimensions)
def fitness(lstm_out,dense_out1,dropout1,activation_dense_1):
    """
    Hyper-parameters:
    lstm_out:  Number of lstm outputs (hidden size)
    dense_outi:   Number of dense layer outputs
    dropouti: Droput rate for dense layer
    activation_dense_i: Activation layer for dense
    """

    # Print the hyper-parameters.
    print('lstm_out:', lstm_out)
    print('dense_out1:', dense_out1)
    print('dropout1:', dropout1)
    print('activation_dense_1:', activation_dense_1)
    print()

    # Create the neural network with these hyper-parameters.
    model = model_builder_ED(lstm_out=lstm_out,dense_out1=dense_out1,dropout1=dropout1,activation_dense_1=activation_dense_1,\
                     nout=n_steps_out,quantile=0.5,n_features=np.shape(xtrain)[-1],n_features_dec=np.shape(xtrain_dec)[-1])

    patience=1 #change this
    callback_log = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    restore_best_weights=True,
                                                    mode='auto')

    # Use Keras to train the model.
    history = model.fit(x=[xtrain,xtrain_dec],
                        y=ytrain,
                        epochs=5, #change this
                        verbose=1,
                        validation_data=([xval,xval_dec],yval),
                        callbacks=[callback_log])

    # Get the prediction accuracy on the validation-set
    # after the last training-epoch.
    predict_model=model.predict(x=[xval,xval_dec]).ravel().reshape((-1,1))
    predict_model=scaler_y.inverse_transform(predict_model)
    y_validi=yval.ravel().reshape((-1,1))
    y_validi=scaler_y.inverse_transform(y_validi)

    # We are using NSE to find the best hyperparamters. This can be set
    # to any other metric.
    accuracy = -NSE(predict_model,y_validi)

    # Print the prediction accuracy.
    print()
    print("Val NSE: {0:.4}".format(-accuracy))
    print()

    # Save the model if it improves on the best-found performance.
    # We use the global keyword so we update the variable outside
    # of this function.
    global best_accuracy

    # If the classification accuracy of the saved model is improved ...
    if accuracy < best_accuracy:

        # Update the classification accuracy.
        best_accuracy = accuracy

    # Delete the Keras model with these hyper-parameters from memory.
    del model

    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()

    # NOTE: Scikit-optimize does minimization so it tries to
    # find a set of hyper-parameters with the LOWEST fitness-value.
    # Because we are interested in the HIGHEST classification
    # accuracy, we need to negate this number so it can be minimized.
    return accuracy

In [ ]:
best_accuracy=10
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=10 ) # change this

In [35]:
# Save results for future use
pd_best=pd.DataFrame(search_result.x,index=['lstm_out','dense_out1','dropout1','activation_dense_1'],columns=['parameter'])
name_best='best_config_%s_lag_%d_lead_%d_ED_v1.csv'%(iid,n_steps_in,n_steps_out)
pd_best.to_csv(name_best)
#files.download(name_best)

In [36]:
search_result.x

[np.int64(254), np.int64(111), 0.15781484223674394, 'relu']

# Retrain the model

In [ ]:
qs = [0.05, 0.5, 0.95]
model_q=[]
for q in qs:
    model=model_builder_ED(lstm_out=search_result.x[0],dense_out1=search_result.x[1],dropout1=search_result.x[2],activation_dense_1=search_result.x[3],\
                     nout=n_steps_out,quantile=0.5,n_features=np.shape(xtrain)[-1],n_features_dec=np.shape(xtrain_dec)[-1])

    model.compile(loss=lambda y,f: tilted_loss(q,y,f), optimizer=Adam(learning_rate=1e-4))
    MAX_EPOCHS = 100 #change this
    patience=10 #change this
    early_stopping_val= tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    restore_best_weights=True,
                                                    mode='auto')
    learning_rate_decrease=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5,\
  mode='auto', min_delta=1e-4, cooldown=0, min_lr=1e-6)
    print('quantile:%1.2f'%(q))
    model.fit(x=[xtrain,xtrain_dec],y=ytrain,epochs=MAX_EPOCHS,
                      validation_data=([xval,xval_dec],yval),
                      callbacks=[early_stopping_val,learning_rate_decrease])
    #model.save('model_%s_lag_%d_lead_%d_quantile_%1.2f_ED.h5'%(iid,n_steps_in,n_steps_out,q))
    model_q.append(model) #list of models